### SQL Queries Notebook

Notebook on SQL queries to construct useful tables stored in dataframes

In [1]:
#load relevant pacakges
import pandas as pd
import numpy as np
import sqlite3 

In [2]:
#connect to datebase and instantiate cursor
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [3]:
#construct match table complete with league and season information
c.execute('''SELECT 
                    q1.id,
                    q1.league,
                    q1.season,
                    q1.date,
                    q1.h_team_name,
                    q1.home_team_goal,
                    q1.away_team_goal,
                    q2.a_team_name
            FROM
            (SELECT 
                   m.id,
                   l.name as league,
                   m.season,
                   m.date,
                   m.match_api_id,
                   m.home_team_api_id,
                   t.team_long_name as h_team_name,
                   m.home_team_goal,
                   m.away_team_goal
            FROM Match m
            LEFT JOIN Team t ON (m.home_team_api_id = t.team_api_id)
            LEFT JOIN League l ON m.league_id = l.id) as q1,
                                  
            (SELECT 
                    mm.id, mm.away_team_api_id, 
                    tt.team_long_name as a_team_name 
            FROM Match mm 
            JOIN Team tt ON mm.away_team_api_id = tt.team_api_id) as q2
                                   
            USING (id)
            ;''')

match_team = pd.DataFrame(c.fetchall())
match_team.columns = [i[0] for i in c.description]

In [4]:
match_team.head(10)

,id,league,season,date,h_team_name,home_team_goal,away_team_goal,a_team_name
0,1,Belgium Jupiler League,2008/2009,2008-08-17 00:00:00,KRC Genk,1,1,Beerschot AC
1,2,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,SV Zulte-Waregem,0,0,Sporting Lokeren
2,3,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,KSV Cercle Brugge,0,3,RSC Anderlecht
3,4,Belgium Jupiler League,2008/2009,2008-08-17 00:00:00,KAA Gent,5,0,RAEC Mons
4,5,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,FCV Dender EH,1,3,Standard de Liège
5,6,Belgium Jupiler League,2008/2009,2008-09-24 00:00:00,KV Mechelen,1,1,Club Brugge KV
6,7,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,KSV Roeselare,2,2,KV Kortrijk
7,8,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,Tubize,1,2,Royal Excel Mouscron
8,9,Belgium Jupiler League,2008/2009,2008-08-16 00:00:00,KVC Westerlo,1,0,Sporting Charleroi
9,10,Belgium Jupiler League,2008/2009,2008-11-01 00:00:00,Club Brugge KV,4,1,KV Kortrijk


In [5]:
#get table with general match information and only include hometeam player1
c.execute('''
            SELECT 
                   id,
                   country_id,
                   league_id,
                   season,
                   date,
                   match_api_id,
                   home_team_api_id,
                   home_player_1
            FROM Match''')

match = pd.DataFrame(c.fetchall())
match.columns = [i[0] for i in c.description]

In [6]:
match.loc[match['home_player_1']==40714].head()

,id,country_id,league_id,season,date,match_api_id,home_team_api_id,home_player_1
4768,4769,4769,4769,2008/2009,2008-08-09 00:00:00,483129,8583,40714.0
4778,4779,4769,4769,2008/2009,2008-10-25 00:00:00,483219,8583,40714.0
4798,4799,4769,4769,2008/2009,2008-11-01 00:00:00,483239,8583,40714.0
4818,4819,4769,4769,2008/2009,2008-11-15 00:00:00,483259,8583,40714.0
4838,4839,4769,4769,2008/2009,2008-11-29 00:00:00,483279,8583,40714.0


In [7]:
#get average player rating for each player
c.execute('''SELECT player_api_id,
                    AVG(overall_rating)
                    FROM Player_Attributes 
                    GROUP BY player_api_id''')

player_attr = pd.DataFrame(c.fetchall())
player_attr.columns = [i[0] for i in c.description]
player_attr.head()

,player_api_id,AVG(overall_rating)
0,2625,60.142857
1,2752,69.380952
2,2768,69.285714
3,2770,71.133333
4,2790,70.200000


In [8]:
#map hometeam player1 rating into match table
c.execute('''SELECT q1.id,
                    q1.country_id,
                    q1.league_id,
                    q1.season,
                    q1.date,
                    q1.match_api_id,
                    q1.home_team_api_id,
                    q2.avg_rating as home_player_1_rating
                    
            FROM(
            

                (SELECT 
                   id,
                   country_id,
                   league_id,
                   season,
                   date,
                   match_api_id,
                   home_team_api_id,
                   home_player_1
                FROM Match) as q1
                
                LEFT JOIN
                
                (SELECT player_api_id,
                    AVG(overall_rating) as avg_rating
                    FROM Player_Attributes 
                    GROUP BY player_api_id) as q2
            
                ON (q1.home_player_1=q2.player_api_id))
               
                
                    ''')

match_player1 = pd.DataFrame(c.fetchall())
match_player1.columns = [i[0] for i in c.description]

In [9]:
match_player1.head()

,id,country_id,league_id,season,date,match_api_id,home_team_api_id,home_player_1_rating
0,1,1,1,2008/2009,2008-08-17 00:00:00,492473,9987,NaN
1,2,1,1,2008/2009,2008-08-16 00:00:00,492474,10000,NaN
2,3,1,1,2008/2009,2008-08-16 00:00:00,492475,9984,NaN
3,4,1,1,2008/2009,2008-08-17 00:00:00,492476,9991,NaN
4,5,1,1,2008/2009,2008-08-16 00:00:00,492477,7947,NaN


In [10]:
match_player1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 8 columns):
id                      25979 non-null int64
country_id              25979 non-null int64
league_id               25979 non-null int64
season                  25979 non-null object
date                    25979 non-null object
match_api_id            25979 non-null int64
home_team_api_id        25979 non-null int64
home_player_1_rating    24755 non-null float64
dtypes: float64(1), int64(5), object(2)
memory usage: 1.6+ MB


In [11]:
match_player1.loc[match_player1['home_player_1_rating'].isnull()]['league_id'].value_counts()

15722    488
13274    214
1        214
17642    142
24558    116
21518     13
4769      13
7809      12
19694     12
Name: league_id, dtype: int64

There are ~1200 missing values for hometeam player1 rating

In [12]:
c.execute('''SELECT * FROM League ''')

league = pd.DataFrame(c.fetchall())
league.columns = [i[0] for i in c.description]

In [13]:
league

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [14]:
#execute query for match home player 2 avg rating
c.execute('''SELECT q1.id,
                    q1.match_api_id,
                    q1.home_team_api_id,
                    q2.avg_rating as home_player_2_rating
                    
            FROM(
                
                (SELECT 
                   id,
                   match_api_id,
                   home_team_api_id,
                   home_player_2
                FROM Match) as q1
                
                LEFT JOIN
                
                (SELECT player_api_id,
                    AVG(overall_rating) as avg_rating
                    FROM Player_Attributes 
                    GROUP BY player_api_id) as q2
                
                ON (q1.home_player_2 = q2.player_api_id))
                            
                    ''')

home_player2_rating = pd.DataFrame(c.fetchall())
home_player2_rating.columns = [i[0] for i in c.description]

In [16]:
home_player2_rating.tail(10)

,id,match_api_id,home_team_api_id,home_player_2_rating
25969,25970,1992086,9931,63.200000
25970,25971,1992087,10192,56.812500
25971,25972,1992088,10199,64.695652
25972,25973,1992089,10243,53.400000
25973,25974,1992090,10179,60.960000
25974,25975,1992091,10190,52.600000
25975,25976,1992092,9824,57.750000
25976,25977,1992093,9956,56.692308
25977,25978,1992094,7896,66.777778
25978,25979,1992095,10192,56.812500


In [17]:
home_player2_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 4 columns):
id                      25979 non-null int64
match_api_id            25979 non-null int64
home_team_api_id        25979 non-null int64
home_player_2_rating    24664 non-null float64
dtypes: float64(1), int64(3)
memory usage: 812.0 KB


currently it's determined that single player ratings can be mapped into match table but mapping all player ratings at once into matches table still need work